<a href="https://colab.research.google.com/github/TeodoroMas/ESM/blob/main/U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

def iou_metric(y_true, y_pred):
  """
  Indice prestazionale per la segmentazione chiamato Intersection-over-Union o Jaccard index.
  IoU = (|X & Y|)/ (|X| + |Y| - |X & Y|)
  """
  yt = y_true[:,:,:,-1:] > 0.5
  yp = y_pred[:,:,:,-1:] > 0.5
  intersection = K.sum(K.cast_to_floatx(yt & yp), axis=(1,2))
  union = K.sum(K.cast_to_floatx(yt | yp), axis=(1,2))
  iou = intersection / K.maximum(union, 0.01)  # the maximum is used to avoid the divisions with zero
  return iou

def fm_metric(y_true, y_pred):
  """
  Indice prestazionale per la segmentazione chiamato F-Measure o Dice coefficeint.
  Fm = (2*|X & Y|)/ ( |X|+ |Y|)
  """
  yt = y_true[:,:,:,-1:] > 0.5
  yp = y_pred[:,:,:,-1:] > 0.5
  intersection = K.sum(K.cast_to_floatx(yt & yp), axis=(1,2))
  denominator  = K.sum(K.cast_to_floatx(yt) + K.cast_to_floatx(yp), axis=(1,2))
  fm = (2*intersection) / K.maximum(denominator, 0.01) # the maximum is used to avoid the divisions with zero
  return fm


def upconv_operation(x, channels, size = (2,2)):
  """
  up-conv: formata dalla seguenza di inperpolazione NN e convoluzione
  """
  x = layers.UpSampling2D(size = size)(x) # Nearest-neighbor interpolation
  x = layers.Conv2D(channels, size, padding = 'same', kernel_initializer = 'he_normal')(x)
  return x

def UNet(input_shape = (512, 512, 3), num_output = 2, fn=64):
  """
  Architettura U-net
  """
  inputs = layers.Input(input_shape)

  #down sampling 
  conv1 = layers.Conv2D(  fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = layers.Conv2D(  fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = layers.MaxPooling2D((2, 2))(conv1)
  conv2 = layers.Conv2D(2*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = layers.Conv2D(2*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = layers.MaxPooling2D((2, 2))(conv2)
  conv3 = layers.Conv2D(4*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = layers.Conv2D(4*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = layers.MaxPooling2D((2, 2))(conv3)
  conv4 = layers.Conv2D(8*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = layers.Conv2D(8*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = layers.Dropout(0.5)(conv4)
  pool4 = layers.MaxPooling2D((2, 2))(drop4)

  #bottleneck
  conv5 = layers.Conv2D(16*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = layers.Conv2D(16*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = layers.Dropout(0.5)(conv5)

  #up sampling
  up6 = upconv_operation(drop5, 8*fn, (2,2))
  merge6 = layers.Concatenate(axis = 3)([drop4,up6])
  conv6 = layers.Conv2D(8*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = layers.Conv2D(8*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = upconv_operation(conv6, 4*fn, (2,2))
  merge7 = layers.Concatenate(axis = 3)([conv3,up7])
  conv7 = layers.Conv2D(4*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = layers.Conv2D(4*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = upconv_operation(conv7, 2*fn, (2,2))
  merge8 = layers.Concatenate(axis = 3)([conv2,up8])
  conv8 = layers.Conv2D(2*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = layers.Conv2D(2*fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = upconv_operation(conv8, fn, (2,2))
  merge9 = layers.Concatenate(axis = 3)([conv1,up9])
  conv9 = layers.Conv2D(  fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = layers.Conv2D(  fn, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  
  if num_output==1:
    activation = 'sigmoid'
  else:
    activation = 'softmax'
  
  conv10 = layers.Conv2D(num_output, (3, 3), activation=activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
  model = keras.models.Model(inputs = inputs, outputs = conv10)
  
  return model


  